In [133]:
from NSEDownload import stocks
import glob
import pandas as pd
import time
from random import randint
import os
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
all_n50_scripts = pd.read_csv('data/nifty_50.csv')

This file contains NIFTY50 index as well. Removing it from the list.

In [3]:
all_n50_scripts = list(all_n50_scripts.Symbol)
all_n50_scripts = all_n50_scripts[1:]

In [4]:
completed_files = os.listdir('data\Adjusted')
completed_files = [file.split('.')[0] for file in completed_files if file.endswith('.csv')]

In [5]:
all_n50_scripts = [script for script in all_n50_scripts if script not in completed_files]
print(len(all_n50_scripts))

0


In [6]:
for script in all_n50_scripts:
    try:
        print("Runnning for {}".format(script))
        df = stocks.get_adjusted_stock(stockSymbol = script, start_date = '1-1-2000', end_date = '18-1-2022')
        df = df.reset_index()
        if df.shape[0] > 1:
            df.to_csv("data\Adjusted\{}.csv".format(script), index=False)
    except:
        print("Not successful for {}".format(script))
        time.sleep(randint(3,10))
        pass
    time.sleep(randint(3,10))

There are changes in symbol for few companies. Let's replace older symbol with latest one and merge all files into one.

In [7]:
file_path = 'data\Adjusted'
completed_files = os.listdir(file_path)
completed_files = [file for file in completed_files if file.endswith('.csv')]

In [8]:
all_scripts_list =[]
for file in completed_files:
    df = pd.read_csv(file_path + '\\' + file)
    if len(df['Symbol'].unique()) > 1:
        latest_symbol = df.tail(1)['Symbol'].values
        df['Symbol'] = latest_symbol[0]
    all_scripts_list.append(df)

In [9]:
all_scripts_df = pd.concat(all_scripts_list, axis=0, ignore_index=True)

This Data needs some cleaning before consumption. Next few steps will aim towards inspecting every column, changing data type(if required) and cleaning it.

In [10]:
all_scripts_df = all_scripts_df.drop_duplicates()

In [11]:
all_scripts_df.Date = pd.to_datetime(all_scripts_df.Date)

In [12]:
all_scripts_df = all_scripts_df.drop(['Series'], axis=1)

In [13]:
all_scripts_df.columns

Index(['Date', 'Symbol', 'Open Price', 'High Price', 'Low Price', 'Last Price',
       'Close Price', 'Average Price', 'Total Traded Quantity', 'Turnover',
       'No. of Trades', 'Deliverable Qty', '% Dly Qt to Traded Qty', ' '],
      dtype='object')

In [14]:
all_scripts_df = all_scripts_df.rename(columns={'Open Price' : 'Open',
'Open Price' : 'Open',
'High Price' : 'High',
'Low Price' : 'Low',
'Last Price' : 'Last',
'Close Price' : 'Close',
'Average Price' : 'Average',
'Total Traded Quantity' : 'Volume',
'No. of Trades' : 'Trades',
'Deliverable Qty' : 'Deliverable_quatity',
'% Dly Qt to Traded Qty' : 'Deliverable_quatity_pct',
})

In [15]:
all_scripts_df = all_scripts_df[['Date', 'Symbol', 'Open', 'High', 'Low', 'Last', 'Close', 'Average',
       'Volume', 'Turnover', 'Trades', 'Deliverable_quatity','Deliverable_quatity_pct']]

In [16]:
all_scripts_df.loc[all_scripts_df.Trades=='-','Trades'] = 0
all_scripts_df['Trades'] = all_scripts_df['Trades'].astype('int')

In [17]:
all_scripts_df.loc[all_scripts_df.Deliverable_quatity=='-','Deliverable_quatity'] = -1
all_scripts_df['Deliverable_quatity'] = all_scripts_df['Deliverable_quatity'].astype('int')

In [18]:
all_scripts_df.loc[all_scripts_df.Deliverable_quatity_pct=='-','Deliverable_quatity_pct'] = -1
all_scripts_df['Deliverable_quatity_pct'] = all_scripts_df['Deliverable_quatity_pct'].astype('float64')

In [19]:
all_scripts_df.loc[all_scripts_df.Trades==0, 'Trades'] = (all_scripts_df[all_scripts_df.Trades==0]['Deliverable_quatity'] * 100) / (all_scripts_df[all_scripts_df.Trades==0]['Deliverable_quatity_pct'] )

In [20]:
all_scripts_df = all_scripts_df.sort_values(by=['Symbol','Date'])

In [22]:
all_scripts_df['10d_max'] = all_scripts_df.groupby('Symbol').rolling(10)['High'].max().groupby(level=0).shift(-10).droplevel(0)

In [23]:
all_scripts_df['10d_pct_change'] = (all_scripts_df['10d_max'] / all_scripts_df['Close'])-1

In [24]:
all_scripts_df = all_scripts_df.drop('10d_max', axis=1)

In [25]:
np.mean(abs(all_scripts_df['10d_pct_change']))

0.06506228528679975

In [26]:
all_scripts_df['10d_pct_change'].describe()

count    239648.000000
mean          0.064564
std           0.064338
min          -0.923041
25%           0.022690
50%           0.047817
75%           0.086067
max           1.126108
Name: 10d_pct_change, dtype: float64

In [27]:
all_scripts_df.to_csv('data/n50_data.csv',index=False)

In [92]:
all_scripts_df['22d_high'] = all_scripts_df.groupby('Symbol').rolling(22)['High'].max().groupby(level=0).shift(-22).droplevel(0)
all_scripts_df['22d_low']= all_scripts_df.groupby('Symbol').rolling(22)['Low'].min().groupby(level=0).shift(-22).droplevel(0)

In [93]:
all_scripts_df.tail()

,Date,Symbol,Open,High,Low,Last,Close,Average,Volume,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct,10d_pct_change,22d_high,22d_low
0,2007-11-27,ADANIPORTS,154.00,210.00,154.0,191.8,192.58,196.944,27294366,2.687719e+10,2.729684e+07,9859619,36.12,0.177692,259.0,168.2
1,2007-11-28,ADANIPORTS,196.80,198.00,174.8,177.0,178.78,188.276,4581338,4.312765e+09,4.581583e+06,1453278,31.72,0.268598,263.8,168.2
2,2007-11-29,ADANIPORTS,181.80,182.95,168.2,177.4,176.84,177.618,5124121,4.550658e+09,5.122979e+06,1069678,20.88,0.300611,264.8,178.0
3,2007-11-30,ADANIPORTS,178.00,191.60,178.0,185.8,184.31,185.834,4609762,4.283257e+09,4.610285e+06,1260913,27.35,0.247898,264.8,184.4
4,2007-12-03,ADANIPORTS,187.95,199.00,184.4,196.0,193.86,193.130,2977470,2.875200e+09,2.977464e+06,816123,27.41,0.204993,264.8,195.2


In [119]:
all_scripts_df[(all_scripts_df['22d_high'] >= 1.1 * all_scripts_df['Close']) 
             & (all_scripts_df['22d_low'] >= 0.97 *  all_scripts_df['Close'])].tail()

,Date,Symbol,Open,High,Low,Last,Close,Average,Volume,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct,10d_pct_change,22d_high,22d_low
240264,2021-12-10,WIPRO,642.15,645.90,635.10,638.40,638.25,639.36,4262606,2.725341e+09,91876.0,2287232,53.66,0.100509,726.8,635.35
240265,2021-12-13,WIPRO,643.50,656.55,640.20,646.95,645.40,648.19,6561840,4.253317e+09,108644.0,3195970,48.71,0.088317,726.8,635.35
240266,2021-12-14,WIPRO,646.00,651.90,639.25,647.00,646.90,647.11,4304706,2.785627e+09,87148.0,1859554,43.20,0.093678,726.8,635.35
240267,2021-12-15,WIPRO,646.70,646.70,635.35,636.55,636.40,638.83,3561772,2.275356e+09,78869.0,1606411,45.10,0.115651,726.8,636.50
240268,2021-12-16,WIPRO,644.00,649.90,639.40,647.50,644.30,644.54,6218620,4.008142e+09,106870.0,2358282,37.92,0.117337,726.8,633.05


In [123]:
all_scripts_df[(all_scripts_df['22d_high'] >= 1.15 * all_scripts_df['Close']) 
             & (all_scripts_df['22d_low'] >= 0.95 *  all_scripts_df['Close'])].tail()

,Date,Symbol,Open,High,Low,Last,Close,Average,Volume,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct,10d_pct_change,22d_high,22d_low
240217,2021-10-01,WIPRO,634.80,639.75,630.20,636.00,636.25,635.17,4208854,2.673343e+09,97991.0,1471892,34.97,0.162829,739.85,634.05
240218,2021-10-04,WIPRO,639.90,643.60,635.95,643.35,641.05,638.84,5247122,3.352074e+09,122772.0,2903508,55.34,0.154122,739.85,634.05
240220,2021-10-06,WIPRO,650.40,650.45,634.40,635.40,635.75,640.73,4836599,3.098930e+09,118313.0,2266980,46.87,0.163744,739.85,635.00
240221,2021-10-07,WIPRO,639.00,648.55,636.90,643.80,642.95,643.86,3724343,2.397969e+09,84656.0,1537354,41.28,0.150712,739.85,635.00
240260,2021-12-06,WIPRO,637.25,644.20,623.15,624.60,624.50,631.19,4704953,2.969720e+09,101023.0,2226869,47.33,0.084868,726.80,623.00


In [129]:
all_scripts_df[['Symbol','Date','Close']].head(11)

,Symbol,Date,Close
0,ADANIPORTS,2007-11-27,192.58
1,ADANIPORTS,2007-11-28,178.78
2,ADANIPORTS,2007-11-29,176.84
3,ADANIPORTS,2007-11-30,184.31
4,ADANIPORTS,2007-12-03,193.86
5,ADANIPORTS,2007-12-04,208.29
6,ADANIPORTS,2007-12-05,216.49
7,ADANIPORTS,2007-12-06,216.26
8,ADANIPORTS,2007-12-07,220.48
9,ADANIPORTS,2007-12-10,215.08


In [132]:
all_scripts_df.groupby('Symbol')['Close'].pct_change(periods=10).hist()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [102]:
all_scripts_df.shape

(240148, 16)

In [ ]:
high>211 & low>=186

In [ ]:
192.58 259 168.2

In [84]:
all_scripts_df.shape

(240148, 16)

In [34]:
np.log(259) - np.log(252.2)

0.02660563816871786

In [39]:
np.log(100) - np.log(75)

0.2876820724517817

In [ ]:
all_scripts_df.High.idxmax

In [73]:
all_scripts_df.groupby('Symbol').rolling(22)['High'].idxmax()

AttributeError: 'RollingGroupby' object has no attribute 'idxmax'

In [41]:
np.log(3.575) - np.log(3.570)

0.0013995803544233354

In [46]:
all_scripts_df.loc[all_scripts_df.reset_index().groupby(['Symbol'].rolling(22))['High'].idxmax()]

AttributeError: 'list' object has no attribute 'rolling'

In [71]:
ap_df = all_scripts_df[all_scripts_df.Symbol=='ADANIPORTS']

In [77]:
ap_df.High.rolling(10).max()

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
3500    774.9
3501    776.0
3502    795.0
3503    795.0
3504    795.0
Name: High, Length: 3502, dtype: float64

In [72]:
ap_df.head()

,Date,Symbol,Open,High,Low,Last,Close,Average,Volume,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct,10d_pct_change,22d_high,22d_low
0,2007-11-27,ADANIPORTS,154.00,210.00,154.0,191.8,192.58,196.944,27294366,2.687719e+10,2.729684e+07,9859619,36.12,0.177692,259.0,252.20
1,2007-11-28,ADANIPORTS,196.80,198.00,174.8,177.0,178.78,188.276,4581338,4.312765e+09,4.581583e+06,1453278,31.72,0.268598,263.8,252.74
2,2007-11-29,ADANIPORTS,181.80,182.95,168.2,177.4,176.84,177.618,5124121,4.550658e+09,5.122979e+06,1069678,20.88,0.300611,264.8,254.00
3,2007-11-30,ADANIPORTS,178.00,191.60,178.0,185.8,184.31,185.834,4609762,4.283257e+09,4.610285e+06,1260913,27.35,0.247898,264.8,254.00
4,2007-12-03,ADANIPORTS,187.95,199.00,184.4,196.0,193.86,193.130,2977470,2.875200e+09,2.977464e+06,816123,27.41,0.204993,264.8,254.00


In [68]:
test_df.head()

,A,B,C,D
2000-01-01,-0.537619,-2.176708,-0.770041,1.277500
2000-01-02,-1.105249,-0.527119,-0.585553,0.797643
2000-01-03,-0.578591,0.108156,-1.951783,0.421310
2000-01-04,-1.582054,-0.682781,-0.508421,-0.514433
2000-01-05,-2.427668,-0.534117,1.368457,1.158240


In [69]:
test_df.groupby('A').idxmax()

,B,C,D
A,,,
-2.576712,2001-08-30,2001-08-30,2001-08-30
-2.560948,2000-11-01,2000-11-01,2000-11-01
-2.539877,2002-05-09,2002-05-09,2002-05-09
-2.492463,2002-04-05,2002-04-05,2002-04-05
-2.463545,2000-03-30,2000-03-30,2000-03-30
...,...,...,...
2.515836,2002-04-02,2002-04-02,2002-04-02
2.709847,2000-07-17,2000-07-17,2000-07-17
2.732829,2000-10-10,2000-10-10,2000-10-10


In [56]:
pd.stats.moments.rolling_apply(test_df,30,lambda x: Series(x).idxmax()).tail(30)

AttributeError: module 'pandas' has no attribute 'stats'

In [60]:
df = pd.DataFrame({'A' : range(5), 'B' : pd.timedelta_range('1 day',periods=5), 'C' : 'foo'})


In [61]:
df

,A,B,C
0,0,1 days,foo
1,1,2 days,foo
2,2,3 days,foo
3,3,4 days,foo
4,4,5 days,foo


In [66]:
df.rolling(2, min_periods=1).idxmax()

AttributeError: 'Rolling' object has no attribute 'idxmax'

In [ ]:
  groupby(keys)[column].idxmax()

In [ ]:
r = df.rolling(2,min_periods=1)
r.agg([np.sum,np.mean])

In [78]:
df = pd.DataFrame(
    {
        "host": ["other", "other", "that", "this", "this"],
        "service": ["mail", "web", "mail", "mail", "web"],
        "no": [1, 2, 1, 2, 1],
    }
).set_index(["host", "service"])

In [80]:
df.reset_index(inplace=True)

In [81]:
df

,host,service,no
0,other,mail,1
1,other,web,2
2,that,mail,1
3,this,mail,2
4,this,web,1


In [82]:
df.groupby('host').agg("idxmax")

,no
host,
other,1
that,2
this,3
